<a href="https://colab.research.google.com/github/yunn333/AI/blob/main/HW0408.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1.引入老師的基礎四套件

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### 2. 安裝並執行 Ollama

先到官網找到安裝程式並安裝。

In [21]:
!curl -fsSL https://ollama.ai/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


###3.啟用ollama伺服器
在伺服器上拉一個
gemma3:1b

In [22]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [23]:
!ollama pull gemma3:1b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠏ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling 7cd4618c1faf... 100% ▕▏ 815 MB                         
pulling e0a42594d802... 100% ▕▏  358 B                         
pulling dd084c7d92a3... 100% ▕▏ 8.4 KB                         
pulling 3116c5225075... 100% ▕▏   77 B                         
pulling 120007c81bf8... 100% ▕▏  492 B                         
verifying sha256 digest 
writing manifest 
success 


### 4. 用 OpenAI API 使用

導入open AI、API金鑰及API網址

In [24]:
import openai
from openai import OpenAI

In [25]:
api_key = "ollama"

In [26]:
client = OpenAI(
    api_key=api_key,
    base_url="http://localhost:11434/v1"
)

### 5. 使用 faiss_db

導入faiss_db相關套件，使用E5 embedding

In [27]:
!wget -O faiss_db.zip 'https://drive.google.com/uc?export=download&id=1i6DZCRcqWtes4_AqzoPWuSBn2qRlZex9'
!unzip faiss_db.zip

--2025-04-21 18:13:08--  https://drive.google.com/uc?export=download&id=1i6DZCRcqWtes4_AqzoPWuSBn2qRlZex9
Resolving drive.google.com (drive.google.com)... 74.125.130.113, 74.125.130.138, 74.125.130.102, ...
Connecting to drive.google.com (drive.google.com)|74.125.130.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1i6DZCRcqWtes4_AqzoPWuSBn2qRlZex9&export=download [following]
--2025-04-21 18:13:09--  https://drive.usercontent.google.com/download?id=1i6DZCRcqWtes4_AqzoPWuSBn2qRlZex9&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 172.253.118.132, 2404:6800:4003:c05::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|172.253.118.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6902 (6.7K) [application/octet-stream]
Saving to: ‘faiss_db.zip’

faiss_db.zip        100%[===================>]   6.74K  --.-KB/s   

In [28]:
!pip install -U langchain langchain-community sentence-transformers faiss-cpu

In [29]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

In [30]:
class CustomE5Embedding(HuggingFaceEmbeddings):
    def embed_documents(self, texts):
        texts = [f"passage: {t}" for t in texts]
        return super().embed_documents(texts)

    def embed_query(self, text):
        return super().embed_query(f"query: {text}")

In [31]:
embedding_model = CustomE5Embedding(model_name="intfloat/multilingual-e5-small")
db = FAISS.load_local("faiss_db", embedding_model, allow_dangerous_deserialization=True)
retriever = db.as_retriever()

### 6. 使用Gradio創造聊天機器人網頁
設定自己的聊天機器人與頁面，並開始應用

In [32]:
!pip install gradio

In [33]:
import gradio as gr

In [34]:
title = "星光引路人"
system_prompt = '''請嚴格完全使用繁體中文回應。你是一位熱愛韓國流行音樂（K-pop）的狂熱粉絲，專門介紹各種不同的韓國偶像團體。
你的職責是根據使用者的喜好，介紹他們未曾了解過的韓團，並提供該團體的詳細資料，包括成員介紹、音樂風格、代表作等，讓使用者更深入了解每個團體的特色。'''

prompt_template = """
根據下列資料回答問題：
{retrieved_chunks}

使用者的問題是：{question}

嚴格使用繁體中文回應。你擁有的資料庫包含各種韓國偶像團體，每個團體的資料都詳細而具娛樂性質。
每個團體的介紹會包括成員的基本資料、音樂風格、經典歌曲、以及他們在音樂界的地位等。
請根據使用者的問題，提供詳盡的回答，並根據使用者的偏好進行個性化推薦，以下是你需要額外提供的資訊：
1. 團體的特色：說明這個團體的特色、為何適合推薦給使用者。
2. 額外提示：可包含團體的熱門歌曲、常見的粉絲活動等（非必須）。

若資料不足以回答問題，請告訴使用者可以稍後再來詢問，看看是否有新的推薦或資料更新。
"""

model = "gemma3:1b"

In [35]:
chat_history = []

def pipi(user_input):
    global chat_history
    # 取回相關資料
    docs = retriever.get_relevant_documents(user_input)
    retrieved_chunks = "\n\n".join([doc.page_content for doc in docs])

    # 將自定 prompt 套入格式
    final_prompt = prompt_template.format(retrieved_chunks=retrieved_chunks, question=user_input)

    # 呼叫 OpenAI API
    response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": final_prompt},
    ]
    )
    answer = response.choices[0].message.content

    chat_history.append((user_input, answer))
    return answer

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("星光引路人")
    gr.Markdown("""
這個 AI 是一個充滿熱情的韓團狂熱粉絲，它會根據使用者的喜好，介紹各種韓國偶像團體。
每次使用者詢問時，AI 都會提供該團體的詳細資料，並根據使用者的需求，給出推薦理由及額外提示。
這些團體可能包括不同風格和時期的韓團，從初出道的小團體到超人氣的明星組合都有。
AI 的目標是讓使用者深入了解 K-pop 的多樣性，並推薦符合他們口味的團體
""")

    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="請輸入你的問題...")

    def respond(message, chat_history_local):
        response = pipi(message)
        chat_history_local.append((message, response))
        return "", chat_history_local

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch(debug=True)

<ipython-input-36-7f932c1f1afd>:10: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://3f063ae151c252020d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
